In [1]:
import numpy as np
import pandas as pd
import pandasql as ps
from pandas import Series, DataFrame
import datetime as dt
import matplotlib.pyplot as plt
import sys
import time

### Отчет по странам с небольшим или отрицательным профитом

In [2]:
import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

In [3]:
sql_srt=\
'''
--Calculate total charge and cost by MCC, MNC, usage_type, account_id
select CONVERT(VARCHAR(8), u.usage_date, 3) as 'REP_DATE'
,u.mcc,u.mnc,u.usage_type,uc.account_id
,sum(u.quantity) as quantity,sum(uc.charge) as charge,sum(uc.cost) as cost from [10028_usage2018-04].[dbo].[USAGE] u
inner join [10028_usage2018-04].[dbo].[USAGE_CHARGES] uc
on u.USAGE_ID = uc.USAGE_ID
group by CONVERT(VARCHAR(8), u.usage_date, 3),u.mcc,u.mnc,u.usage_type,uc.account_id
order by u.mcc,CONVERT(VARCHAR(8), u.usage_date, 3)
'''

In [4]:
df = pd.read_sql_query(sql_srt, cnxn,parse_dates={'REP_DATE': '%d/%m/%y'})
df.head()

,REP_DATE,mcc,mnc,usage_type,account_id,quantity,charge,cost
0,2018-04-01,202,5,15,352321.0,4.05000,0.00000,0.00000
1,2018-04-01,202,5,15,352370.0,278.61663,0.00000,0.00000
2,2018-04-01,202,5,21,352321.0,18.00000,0.00000,0.00000
3,2018-04-01,202,5,21,352334.0,26.00000,0.00000,0.00000
4,2018-04-01,202,5,22,352334.0,63.00000,0.63945,0.37044


In [5]:
def f(x):
    data=x[['charge','cost']]
    x['profit']=data['charge']-data['cost']
    if data['cost']!=0:
        x['profit_per']=round((x['profit']/data['cost'])*100,1)
    else:
        x['profit_per']=0
    return x

df1=df.groupby(['mcc','mnc']).agg({'charge': np.sum, 'cost': np.sum}).apply(f,axis=1).copy()

In [6]:
df1[df1['profit']<0]

charge         cost      profit  profit_per
mcc mnc                                                 
208 10      2.37834     34.06715   -31.68881       -93.0
    29    233.39442    274.69781   -41.30339       -15.0
214 9      24.09512    676.18343  -652.08831       -96.4
234 3       0.27545      0.91020    -0.63475       -69.7
240 10      0.07635      0.14915    -0.07280       -48.8
310 260    10.61740    677.87273  -667.25533       -98.4
    680  1465.88595   2125.99344  -660.10749       -31.0
515 2    6012.19448  10894.22103 -4882.02655       -44.8
901 18      0.74520      8.12001    -7.37481       -90.8

In [7]:
df2=df.groupby(['REP_DATE','mcc','mnc']).agg({'charge': np.sum, 'cost': np.sum}).apply(f,axis=1).copy()
df2.head()

,,,charge,cost,profit,profit_per
REP_DATE,mcc,mnc,,,,
2018-02-16,530,1,0.00026,0.00015,0.00011,73.3
2018-02-21,222,10,0.78593,0.45280,0.33313,73.6
2018-02-22,222,10,0.28601,0.16414,0.12187,74.2
2018-02-23,222,10,0.05382,0.02994,0.02388,79.8
2018-02-24,222,10,0.09339,0.05312,0.04027,75.8


In [9]:
df2[(df2.index.get_level_values('mcc')==310) & (df2.index.get_level_values('mnc')==680)].head()

,,,charge,cost,profit,profit_per
REP_DATE,mcc,mnc,,,,
2018-04-01,310,680,78.21434,72.55151,5.66283,7.8
2018-04-02,310,680,42.00338,53.25511,-11.25173,-21.1
2018-04-03,310,680,64.79633,61.03443,3.76190,6.2
2018-04-04,310,680,173.01345,144.49367,28.51978,19.7
2018-04-05,310,680,29.24078,47.55932,-18.31854,-38.5
